<a href="https://colab.research.google.com/github/Walbyq/data-science/blob/main/kc-house/kc_house.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импорт библиотек

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import warnings

# Подготовка данных

Подключаемся к Google Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Получаем датафрейм:

In [ ]:
csv_file_path = '/content/drive/MyDrive/ML/datasets/kc_house_data.csv'
df = pd.read_csv(csv_file_path)

Ссылка на набор данных: https://www.kaggle.com/datasets/harlfoxem/housesalesprediction

Удаляем аномальное значение о доме с 33 спальными комнатами:

In [ ]:
df = df[df['bedrooms'] != 33]

# Анализ данных

## Основная информация

Столбцы датафрейма:

*   date - дата продажи;
*   price	- стоимость продажи;
*   bedrooms	- количество спальных комнат;
*   bathrooms	- количество ванных комнат;
*   sqft_living	- жилая площадь (в квадратных футах);
*   sqft_lot	- площадь участка (в квадратных фунтах);
*   floors	- количество этажей;
*   waterfront	- расположение на набережной (0 - нет выхода к воде, 1 - есть);
*   view	- качество обзора (0-4);
*   condition	- состояние дома (1-5);
*   grade	- качество строительства: качество материалов и выполнения работ (1-13);
*   sqft_above	- площадь над землей (в квадратных футах);
*   sqft_basement	- площадь подвала (в квадратных футах);
*   yr_built	- дата постройки;
*   yr_renovated	- дата последнего капитального ремонта (0 - никогда не было);
*   zipcode	- почтовый индекс;
*   lat	- широта;
*   long	- долгота;
*   sqft_living15	- средний размер жилой площади в ближайших 15 домах (в квадратных футах);
*   sqft_lot15 - средний размер площади участка в ближайших 15 домах (в квадратных фунтах).

Выводим размерность:

In [ ]:
df.shape

Выводим первые 5 значений:

In [ ]:
df.head()

Выводим основные статистические характеристики данных по каждому числовому признаку:

In [ ]:
df.describe()

Выводим основную информацию:

In [ ]:
df.info()

Выводим количество пустых значений в каждом столбце:

In [ ]:
df.isnull().sum()

## Парные диаграммы

Визуализируют взаимосвязи между парами переменных в наборе данных с помощью диаграмм рассеяния и гистограмм.

In [ ]:
sns.pairplot(df, size=1.3)

## Тепловая карта по матрице корреляции

Корреляция указывает на степень взаимосвязи между двумя переменными. Она помогает понять, как одна переменная изменяется по отношению к другой. Значение варьируется от -1 до 1, где:

*   1 указывает на полную положительную корреляцию;
*   -1 указывает на полную отрицательную корреляцию;
*   0 указывает на отсутствие корреляции.

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(numeric_only=True), cmap='coolwarm')

## Гистограммы распределения и диаграммы размаха

Левая колонка графиков показывает гистограммы распределения для каждой характеристики. Они отображают, как часто встречаются различные значения каждой характеристики в наборе данных.

Правая колонка графиков показывает диаграммы размаха, которые визуализируют связь между каждой характеристикой и ценой недвижимости.

In [ ]:
features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
        'waterfront', 'view', 'condition', 'grade', 'sqft_above',
        'sqft_basement', 'yr_built', 'yr_renovated', 'lat', 'long',
        'sqft_living15', 'sqft_lot15']

fig, axes = plt.subplots(len(features), 2, figsize=(15, 80))

for i, feature in enumerate(features):
    # Создаем подграфик для каждой пары
    subfig = fig.add_subplot(len(features), 1, i+1)
    subfig.set_title(f'Параметр {feature}', fontsize=16, y=1.15)
    subfig.axis('off')  # Отключаем оси для подграфика

    # Гистограмма
    sns.histplot(df[feature], ax=axes[i, 0])

    # Диаграмма размаха
    if feature in ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement',
               'yr_built', 'lat', 'long', 'sqft_living15', 'sqft_lot15']:
        # Площадь делим на промежутки для наглядности
        df['group'] = pd.cut(df[feature], bins=20)
        sns.boxplot(df, x='group', y='price', ax=axes[i, 1])

    elif feature == 'yr_renovated':
        # Создаем бинарный признак для renovated
        df['renovated_binary'] = (df['yr_renovated'] > 0).astype(int)
        sns.boxplot(df, x='renovated_binary', y='price', ax=axes[i, 1])

    else:
        sns.boxplot(df, x=feature, y='price', ax=axes[i, 1])

    # Ограничение числа подписей для диаграммы размаха
    x_ticks = axes[i, 1].get_xticks()
    if len(x_ticks) >= 20: axes[i, 1].set_xticks(x_ticks[::4])

plt.tight_layout()
plt.show()

## Точечный 3D график распределения признаков

Показывает распределение цен на недвижимость в зависимости от географического расположения домов.

In [ ]:
fig = px.scatter_3d(df, x='lat', y='long', z='price', color='price',
                    title='3D распределение цен в зависимости от расположения дома')
fig.show()

## Вывод

На основе проведенного анализа можно сделать следующие выводы:

*   Площадь жилых помещений (sqft_living) имеет сильную положительную корреляцию с ценой;
*   Качество строительства (grade) также значительно влияет на стоимость дома;
*   Количество спален и ванных комнат в целом положительно коррелирует с ценой, но эта связь не линейна;
*   Расположение дома (lat, long) играет важную роль в формировании цены;
*   Дома с видом на воду (waterfront) обычно стоят дороже;
*   Более новые дома и недавно отремонтированные, как правило, имеют более высокую стоимость.

# Предобработка данных

Объединим значения по периодам в признаках даты постройки и даты последнего капитального ремонта:

In [ ]:
def get_period(year):
  if year < 1950:
      return 'До 1950'
  elif 1950 <= year < 1980:
      return '1950-1979'
  elif 1980 <= year < 2000:
      return '1980-1999'
  else:
      return 'После 2000'

df['build_period'] = df['yr_built'].apply(get_period)
df['renovated_period'] = df['yr_renovated'].apply(lambda x: get_period(x) if x != 0 else 'Не ремонтировался')

# Нормализация числовых данных
numeric_features = ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'lat', 'long']
scaler = StandardScaler()
df[numeric_features] = scaler.fit_transform(df[numeric_features])

# Присваиваем переменные с признаками и целевую переменную
X, y = df[features], df['price']

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабируем признаки
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение

Обучение моделей:

In [ ]:
# Игнорирование предупреждений
warnings.filterwarnings("ignore")

# Словарь с моделями
models = {
    "Linear Regression": LinearRegression(),
    "Lasso Regression": Lasso(alpha=1.0),
    "Ridge Regression": Ridge(alpha=1.0),
    "Polynomial Regression": make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
}

results = []

for name, model in models.items():
    # Обучение модели
    model.fit(X_train, y_train)

    # Предсказание на тестовой выборке
    y_pred = model.predict(X_test)

    # Расчет метрик
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    # Сохранение результатов
    results.append({
        "Model": name,
        "R-squared": r2,
        "MAE": mae,
        "MSE": mse
    })

Вывод результатов обучения:

In [ ]:
# Вывод точности обучения
print("Точность обучения моделей моделей:")
print(pd.DataFrame(results))